In [ ]:
%% matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import itertools

In [ ]:
# loading data
data = pd.read_excel('Data/PERM_Disclosure_Data_FY15_Q4.xlsx', 'DDFY2015_Final')

In [10]:
# removing withdrawn applications
data = data[data['CASE_STATUS'] != 'Withdrawn']

In [11]:
# adding a binary series with 1 indicating that an application was denied
data['denied'] = (data.CASE_STATUS == 'Denied').astype(int)

Descriptive Statistics

In [12]:
data['CASE_STATUS'].value_counts()

Certified            40176
Certified-Expired    38762
Denied                5696
Name: CASE_STATUS, dtype: int64

In [13]:
pcert, pcertex, pdenied = (data['CASE_STATUS'].value_counts().values) / float(len(data))
size = len(data)
print 'Certified: \t\t', pcert, '\nCertified-Expired: \t', pcertex, '\nDenied: \t\t', pdenied, '\nSize: \t\t\t', size
                                                                 

Certified: 		0.474702838103 
Certified-Expired: 	0.457995604603 
Denied: 		0.0673015572938 
Size: 			84634


In [14]:
serror = (pdenied*(1-pdenied)/float(len(data))) ** .5
print 'Standard Error: ',serror

Standard Error:  0.000861213341343


Categorical Data

In [15]:
# possibly interesting columns of categorical data
dfcat = data[['denied', 'EMPLOYER_NAME', 'EMPLOYER_STATE', 'AGENT_FIRM_NAME', 'PW_SOC_TITLE', 'JOB_INFO_EDUCATION', 
              'COUNTRY_OF_CITIZENSHIP', 'CLASS_OF_ADMISSION', 'FOREIGN_WORKER_INFO_EDUCATION', 
              'FOREIGN_WORKER_INFO_MAJOR']]

After exploring the columns, Visa Type, Country of Citizenship, and Education seem to be the most interesting columns.

In [16]:
# visa types with a high rate of denial
dftype = dfcat[['CLASS_OF_ADMISSION', 'denied']]
dftype1 = dftype.groupby('CLASS_OF_ADMISSION').mean()
dftype1['size'] = dftype.groupby('CLASS_OF_ADMISSION').agg(sum)
dftype = dftype1[(dftype1['denied'] > (pdenied + 2*serror)) & (dftype1['size'] >= 10)]
bad_visa = [str(x) for x in dftype.index]

In [17]:
# countries with a high rate of denial
dfcit = dfcat[['COUNTRY_OF_CITIZENSHIP', 'denied']]
dfcit1 = dfcit.groupby('COUNTRY_OF_CITIZENSHIP').mean()
dfcit1['size'] = dfcit.groupby('COUNTRY_OF_CITIZENSHIP').agg(sum)
dfcit = dfcit1[(dfcit1['denied'] > (pdenied + 2*serror)) & (dfcit1['size'] >= 10)]
bad_con = [str(x) for x in dfcit.index]

In [18]:
dfcat.groupby(['FOREIGN_WORKER_INFO_EDUCATION'])['denied'].mean().sort_values()

FOREIGN_WORKER_INFO_EDUCATION
Doctorate      0.033010
Master's       0.051275
Bachelor's     0.053348
Other          0.074332
Associate's    0.185270
None           0.228803
High School    0.270021
Name: denied, dtype: float64

In [19]:
dfcat.groupby(['JOB_INFO_EDUCATION'])['denied'].mean().sort_values()

JOB_INFO_EDUCATION
Doctorate      0.025573
Master's       0.047168
Bachelor's     0.051324
Other          0.075218
Associate's    0.188406
None           0.233772
High School    0.304193
Name: denied, dtype: float64

In [20]:
# educational requirements/attainment with a low level of denial
good_ed = ['Bachelor\'s', 'Master\'s', 'Doctorate']

In [21]:
# adding new columns converting categorical data into binary
data['bad_country'] = [int(str(x) in bad_con) for x in data.COUNTRY_OF_CITIZENSHIP.values]
data['worker_good_ed'] = [int(str(x) in good_ed) for x in data.FOREIGN_WORKER_INFO_EDUCATION.values]
data['job_good_ed'] = [int(str(x) in good_ed) for x in data.JOB_INFO_EDUCATION.values]
data['visa_type'] = [int(str(x) in bad_visa) for x in data.CLASS_OF_ADMISSION.values]

In [22]:
# interesting fields with binary or numerical data
data_fields = ['REFILE', 'EMPLOYER_NUM_EMPLOYEES', 
               'FOREIGN_WORKER_OWNERSHIP_INTEREST', 'PW_AMOUNT_9089', 'WAGE_OFFER_FROM_9089', 'WAGE_OFFER_TO_9089', 
               'JOB_INFO_TRAINING', 'JOB_INFO_TRAINING_NUM_MONTHS', 'JOB_INFO_EXPERIENCE', 
               'JOB_INFO_EXPERIENCE_NUM_MONTHS', 'JOB_INFO_ALT_FIELD', 'JOB_INFO_ALT_COMBO_ED_EXP', 
               'JOB_INFO_ALT_CMB_ED_OTH_YRS', 'JOB_INFO_FOREIGN_ED', 'JOB_INFO_ALT_OCC', 'JOB_INFO_ALT_OCC_NUM_MONTHS', 
               'JOB_INFO_JOB_REQ_NORMAL', 'JOB_INFO_FOREIGN_LANG_REQ', 'JOB_INFO_COMBO_OCCUPATION',
               'JI_FOREIGN_WORKER_LIVE_ON_PREMISES', 'JI_LIVE_IN_DOMESTIC_SERVICE', 'JI_LIVE_IN_DOM_SVC_CONTRACT', 
               'RECR_INFO_PROFESSIONAL_OCC', 'RECR_INFO_COLL_UNIV_TEACHER', 'RECR_INFO_COLL_TEACH_COMP_PROC', 
               'RI_POSTED_NOTICE_AT_WORKSITE', 'RI_LAYOFF_IN_PAST_SIX_MONTHS', 'RI_US_WORKERS_CONSIDERED', 
               'FOREIGN_WORKER_INFO_TRAINING_COMP', 'FOREIGN_WORKER_INFO_REQ_EXPERIENCE', 
               'FOREIGN_WORKER_INFO_ALT_EDU_EXPERIENCE', 'FOREIGN_WORKER_INFO_REL_OCCUP_EXP', 
               'PREPARER_INFO_EMP_COMPLETED']
dflogit = data[data_fields]

In [23]:
def remove_commas(uni):
    # converts unicode numbers with commas into floats
    string = str(uni)
    return float(string.replace(',', ''))

def data_clean(df):
    # formats datatypes to binary for indicators or floats for numbers
    columns = df.columns
    dfr = pd.DataFrame()
    for column in columns:
        name = str(column)
        name = name.lower()
        if str(df[column][0]) in ['Y', 'N']:
            dfr[name] = (df[column] == 'Y').astype(int)
        elif ',' in str(df[column][0]):
            try:
                dfr[name] = df[column].apply(remove_commas)
            except ValueError:
                continue
        else:
            try:
                dfr[name] = df[column].apply(float)
            except ValueError:
                continue
    return dfr

def add_columns(df1, df2, columns):
    # adds columns from one dataframe to another
    for column in columns:
        df2[column] = df1[column]

In [24]:
# formatting data for logistic regression
dflogit = data_clean(dflogit)
new_columns = ['bad_country', 'worker_good_ed', 'job_good_ed', 'visa_type', 'denied']
add_columns(data, dflogit, new_columns)
dflogit.head()

,refile,employer_num_employees,foreign_worker_ownership_interest,pw_amount_9089,job_info_training,job_info_training_num_months,job_info_experience,job_info_experience_num_months,job_info_alt_field,job_info_alt_combo_ed_exp,...,recr_info_coll_univ_teacher,ri_posted_notice_at_worksite,ri_layoff_in_past_six_months,foreign_worker_info_rel_occup_exp,preparer_info_emp_completed,bad_country,worker_good_ed,job_good_ed,visa_type,denied
0,0,1935,0,83366,0,NaN,0,NaN,0,0,...,0,1,0,1,0,0,1,1,0,0
1,0,350,0,16973,0,NaN,0,NaN,0,0,...,0,1,0,0,0,1,0,0,0,1
2,0,4,0,49982,0,NaN,1,36,0,0,...,0,1,0,0,0,0,1,1,0,1
3,0,8,0,43514,0,NaN,0,NaN,0,0,...,0,1,0,0,0,1,1,1,1,0
4,0,350,0,16973,0,NaN,0,NaN,0,0,...,0,1,0,0,0,1,0,0,0,1


In [25]:
# checking data types
dflogit.dtypes

refile                                  int64
employer_num_employees                float64
foreign_worker_ownership_interest       int64
pw_amount_9089                        float64
job_info_training                       int64
job_info_training_num_months          float64
job_info_experience                     int64
job_info_experience_num_months        float64
job_info_alt_field                      int64
job_info_alt_combo_ed_exp               int64
job_info_alt_cmb_ed_oth_yrs           float64
job_info_foreign_ed                     int64
job_info_alt_occ                        int64
job_info_alt_occ_num_months           float64
job_info_job_req_normal                 int64
job_info_foreign_lang_req               int64
job_info_combo_occupation               int64
ji_foreign_worker_live_on_premises      int64
ji_live_in_domestic_service             int64
recr_info_professional_occ              int64
recr_info_coll_univ_teacher             int64
ri_posted_notice_at_worksite      

In [107]:
# performing 1 parameter logistic regression to ID independent variables with the strongest relationships 
# with the dependent variable
params = dflogit.columns[:-1]
rs = {}
for column in params:
    model = smf.logit('denied ~ ' + column, data=dflogit)
    results = model.fit()
    rs[results.prsquared] = column

Optimization terminated successfully.
         Current function value: 0.245956
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.245351
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.245739
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.231202
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.245884
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.327335
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.245096
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.210865
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.244128
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.244154
  

In [108]:
# ranking parameters by relationship with the dependent variable
ranked_params = []
for key in sorted(rs.keys())[::-1]:
    ranked_params.append(rs[key])

In [109]:
# formatting parameters for logit function
test_params = ' + '.join(ranked_params[:19])

In [110]:
# running the regression
model = smf.logit('denied ~ ' + test_params, data=dflogit)
results = model.fit(maxiter=10000)
results.summary()

Optimization terminated successfully.
         Current function value: 0.199373
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 denied   No. Observations:                 1742
Model:                          Logit   Df Residuals:                     1722
Method:                           MLE   Df Model:                           19
Date:                Thu, 29 Oct 2015   Pseudo R-squ.:                  0.4549
Time:                        16:34:07   Log-Likelihood:                -347.31
converged:                       True   LL-Null:                       -637.11
                                        LLR p-value:                1.013e-110
======================================================================================================
                                         coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------------
Intercept                              1.0995      0.864      1.272      0.203        -0.594     2.793
job_info_training_num_months          -0.0108      0.005     -1.968      0.049        -0.022 -4.51e-05
job_good_ed                           -1.2478      0.669     -1.864      0.062        -2.560     0.064
pw_amount_9089                     -8.721e-06   2.32e-06     -3.752      0.000     -1.33e-05 -4.17e-06
recr_info_professional_occ            -1.1153      0.271     -4.109      0.000        -1.647    -0.583
worker_good_ed                         1.9735      0.649      3.041      0.002         0.702     3.245
job_info_foreign_ed                   -0.7882      0.309     -2.553      0.011        -1.393    -0.183
bad_country                            0.5433      0.215      2.522      0.012         0.121     0.966
job_info_job_req_normal                0.2876      0.710      0.405      0.686        -1.105     1.680
job_info_alt_occ                      -0.2406      0.494     -0.487      0.626        -1.209     0.728
foreign_worker_info_rel_occup_exp     -0.0470      0.496     -0.095      0.925        -1.019     0.926
visa_type                              0.8027      0.279      2.879      0.004         0.256     1.349
ri_posted_notice_at_worksite          -2.1751      0.399     -5.449      0.000        -2.958    -1.393
job_info_alt_field                    -0.2131      0.352     -0.606      0.545        -0.903     0.477
job_info_alt_combo_ed_exp              0.1569      0.380      0.413      0.680        -0.588     0.902
job_info_experience                    1.0359      0.249      4.154      0.000         0.547     1.525
ji_foreign_worker_live_on_premises     2.2529      1.147      1.965      0.049         0.006     4.500
job_info_foreign_lang_req              1.4474      0.436      3.321      0.001         0.593     2.302
job_info_combo_occupation              0.2192      0.362      0.606      0.545        -0.490     0.929
foreign_worker_ownership_interest      1.8633      1.252      1.488      0.137        -0.591     4.318
======================================================================================================
"""